# Data Loading, Cleaning & Export

### Importazione Librerie

In [52]:
import numpy as np
import pandas as pd
import requests

### Caricamento Dati

In [54]:
CSV_files = {
    "customers":           r"dataset/olist_customers_dataset.csv",
    "geolocation_dataset": r"dataset/olist_geolocation_dataset.csv",
    "order_items":         r"dataset/olist_order_items_dataset.csv",
    "order_payments":      r"dataset/olist_order_payments_dataset.csv",
    "order_review":        r"dataset/olist_order_reviews_dataset.csv",
    "order_dataset":       r"dataset/olist_orders_dataset.csv",
    "list_product":        r"dataset/olist_products_dataset.csv",
    "list_seller":         r"dataset/olist_sellers_dataset.csv",
    "product_category":    r"dataset/product_category_name_translation.csv"
}

dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

df_customers           = dataframes["customers"]
df_orders_items        = dataframes["order_items"]
df_order_payments      = dataframes["order_payments"]
df_order_review        = dataframes["order_review"]
df_order_dataset       = dataframes["order_dataset"]
df_list_product        = dataframes["list_product"]
df_list_seller         = dataframes["list_seller"]
df_product_category    = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]

### Analisi Preliminare

In [56]:
# Stampa un riepilogo di ogni csv con la sua shape, i missing values e i duplicati.
def quick_overview(df, name):

    # Nome DF
    print(f"{name}")
    print(f"Shape: {df.shape}")

    # Valori nulli
    nulls = df.isna().sum()
    nulls = nulls[nulls > 0]
    if len(nulls) > 0:
        print(f"Valori nulli:\n{nulls.to_string()}\n")
    else:
        print("Nessun valore nullo\n")

for name, df in dataframes.items():
    quick_overview(df, name)

customers
Shape: (99441, 5)
Nessun valore nullo

geolocation_dataset
Shape: (1000163, 5)
Nessun valore nullo

order_items
Shape: (112650, 7)
Nessun valore nullo

order_payments
Shape: (103886, 5)
Nessun valore nullo

order_review
Shape: (99224, 7)
Valori nulli:
review_comment_title      87656
review_comment_message    58247

order_dataset
Shape: (99441, 8)
Valori nulli:
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965

list_product
Shape: (32951, 9)
Valori nulli:
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2

list_seller
Shape: (3095, 4)
Nessun valore nullo

product_category
Shape: (71, 2)
Nessun valore nullo



### PULIZIA: df_list_product

In [57]:
# 1) Droppiamo le colonne (peso, misure)
# 2) Droppiamo i 610 prodotti senza categoria (< 2% del totale):
#    verranno esclusi anche dagli altri df.
# 3) Uniamo la traduzione inglese della categoria.

# 1
df_list_product = df_list_product.drop(
    columns=['product_name_lenght', 'product_width_cm',
             'product_height_cm', 'product_length_cm', 'product_weight_g']
)

# 2
# Salviamo gli id dei prodotti senza categoria per filtrare order_items dopo
product_id_nan = df_list_product[df_list_product['product_category_name'].isna()]
product_id_to_delete = product_id_nan['product_id'].to_numpy()
df_list_product = df_list_product.dropna(subset=['product_category_name'])

# 3
df_list_product = (
    df_list_product
    .merge(df_product_category, on='product_category_name', how='left')
)

df_list_product.info()
df_list_product.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 32341 entries, 0 to 32340
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32341 non-null  str    
 1   product_category_name          32341 non-null  str    
 2   product_description_lenght     32341 non-null  float64
 3   product_photos_qty             32341 non-null  float64
 4   product_category_name_english  32328 non-null  str    
dtypes: float64(2), str(3)
memory usage: 1.2 MB


,product_id,product_category_name,product_description_lenght,product_photos_qty,product_category_name_english
19021,e7d7558873ff13253317bd0aac27f072,telefonia,382.0,10.0,telephony
18422,43e6aab5f2602eb0af739f270de920d5,esporte_lazer,310.0,1.0,sports_leisure


In [61]:
# Traduciamo con Selenium le categorie portoghesi senza traduzione inglese
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Troviamo i nomi portoghesi che non hanno traduzione nel dataframe df_product_category
traduzione_mancante = df_list_product['product_category_name_english'].isna()
nomi_mancanti = df_list_product[traduzione_mancante]['product_category_name'].unique()
print(f"Categorie da tradurre ({len(nomi_mancanti)}): \n{nomi_mancanti}\n")


Categorie da tradurre (2): 
<StringArray>
['pc_gamer', 'portateis_cozinha_e_preparadores_de_alimentos']
Length: 2, dtype: str



In [62]:
# Apre Google Translate nel browser e restituisce la traduzione
# dal portoghese all'inglese del testo passato, formattata in snake_case.
def traduci_categoria(testo):

    # Costruisce l'URL di Google Translate con i parametri: lingua portoghese (pt), lingua inglese (en) e testo
    url = f"https://translate.google.com/?sl=pt&tl=en&text={testo}&op=translate"
    driver.get(url)
    
    # Attende 2 secondi che la pagina carichi e la traduzione appaia
    time.sleep(2)

    # Accetta i cookie se il popup è presente
    try:
        accept_button = driver.find_element(By.XPATH, "//button[.//span[text()='Accept all']]")
        accept_button.click()
        time.sleep(2)
    except:
        pass  # Il popup non è presente, continua normalmente
    
    # Trova l'elemento HTML che contiene il testo tradotto
    risultato = driver.find_element(By.XPATH, "//span[@jsname='W297wb']")
    
    # Converte in minuscolo e sostituisce gli spazi con underscore
    traduzione = risultato.text.lower().replace(' ', '_')
    return traduzione

# Avvia Firefox
# NB CONTROLLATE TUTTI SE SI APRE IL BROWSER E VEDETE GOOGLE TRANSLATE
driver = webdriver.Firefox()

# Dizionario che conterrà le traduzioni: {nome_portoghese: traduzione_inglese}
traduzioni_manuali = {}

# Itera sui nomi di categoria mancanti di traduzione
for nome in nomi_mancanti:
    traduzione = traduci_categoria(nome)
    traduzioni_manuali[nome] = traduzione
    print(f"{nome}  →  {traduzione}")
    time.sleep(1)

# Chiude il browser al termine delle traduzioni
driver.quit()

# Aggiorna il DataFrame: per ogni categoria tradotta,
# riempie i valori NaN nella colonna 'product_category_name_english'
for nome, traduzione in traduzioni_manuali.items():
    df_list_product.loc[
        df_list_product['product_category_name'] == nome,
        'product_category_name_english'
    ] = traduzione

df_list_product.info()

pc_gamer  →  gaming_pc
portateis_cozinha_e_preparadores_de_alimentos  →  portateis_kitchen_e_food_preparators
<class 'pandas.DataFrame'>
RangeIndex: 32341 entries, 0 to 32340
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32341 non-null  str    
 1   product_category_name          32341 non-null  str    
 2   product_description_lenght     32341 non-null  float64
 3   product_photos_qty             32341 non-null  float64
 4   product_category_name_english  32341 non-null  str    
dtypes: float64(2), str(3)
memory usage: 1.2 MB


### PULIZIA: df_orders_items

In [63]:
url = 'https://v6.exchangerate-api.com/v6/ee8e65018f5adf36f58283bd/latest/EUR'
oae = requests.get(url)
print(oae.status_code)#richiesta API  

200


In [64]:
EUR_BRL=oae.json()['conversion_rates']['BRL']#Stanziata la variabile del tasso di cambio chiamndolo EUR_BRL
EUR_BRL#tasso conversione

6.1292

In [65]:
# 1) Escludiamo gli items con product_id in product_id_to_delete
# 2) Convertiamo shipping_limit_date in datetime

# 1
df_orders_items = (
    df_orders_items[~df_orders_items['product_id'].isin(product_id_to_delete)]
    .copy()
    .reset_index(drop=True)
)

# 2
df_orders_items['shipping_limit_date'] = pd.to_datetime(
    df_orders_items['shipping_limit_date']
)

#conversione in euro di price e freight_value
df_orders_items['eur_price']=round(df_orders_items['price']/EUR_BRL,2)
df_orders_items['eur_freight_value']=round(df_orders_items['freight_value']/EUR_BRL,2)
#rimozione colonne in real breasiliano
df_orders_items=df_orders_items.drop(['price','freight_value'],axis=1)


df_orders_items.info()
df_orders_items.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 111047 entries, 0 to 111046
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             111047 non-null  str           
 1   order_item_id        111047 non-null  int64         
 2   product_id           111047 non-null  str           
 3   seller_id            111047 non-null  str           
 4   shipping_limit_date  111047 non-null  datetime64[us]
 5   eur_price            111047 non-null  float64       
 6   eur_freight_value    111047 non-null  float64       
dtypes: datetime64[us](1), float64(2), int64(1), str(3)
memory usage: 5.9 MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,eur_price,eur_freight_value
33152,4c3302743a4cdbc7c65bb027ea156c35,1,76b7fa5da50b22fe17d1e7bbde661106,2a261b5b644fa05f4f2700eb93544f2c,2018-03-07 18:49:08,7.34,2.87
44923,67796dbbc55a23910220b90d80294285,1,3410cbd7df2130ec3cb9300ef0fe3df1,165fc07beebdcb6190fba8a06db2a449,2017-09-21 02:50:10,9.63,2.64


### PULIZIA: df_order_dataset

In [67]:
# 1) Conversione delle colonne data in datetime

# 2) delivery_delay_days : differenza tra consegna effettiva e stimata
#    (positivo = in ritardo, negativo = in anticipo)
#    actual_delivery_days: giorni totali dall'acquisto alla consegna

# 1
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    df_order_dataset[col] = pd.to_datetime(df_order_dataset[col])

# 2
df_order_dataset['delivery_delay_days'] = (
    df_order_dataset['order_delivered_customer_date'] -
    df_order_dataset['order_estimated_delivery_date']
).dt.days
df_order_dataset['actual_delivery_days'] = (
    df_order_dataset['order_delivered_customer_date'] -
    df_order_dataset['order_purchase_timestamp']
).dt.days

df_order_dataset.info()
df_order_dataset.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  str           
 1   customer_id                    99441 non-null  str           
 2   order_status                   99441 non-null  str           
 3   order_purchase_timestamp       99441 non-null  datetime64[us]
 4   order_approved_at              99281 non-null  datetime64[us]
 5   order_delivered_carrier_date   97658 non-null  datetime64[us]
 6   order_delivered_customer_date  96476 non-null  datetime64[us]
 7   order_estimated_delivery_date  99441 non-null  datetime64[us]
 8   delivery_delay_days            96476 non-null  float64       
 9   actual_delivery_days           96476 non-null  float64       
dtypes: datetime64[us](5), float64(2), str(3)
memory usage: 7.6 MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,actual_delivery_days
87214,b40a0752445a5228f3063cc30201bb11,15b359e85cd990ffbc6e10ef35ded3fb,delivered,2018-01-17 18:11:31,2018-01-17 18:19:22,2018-01-19 19:46:49,2018-01-26 19:55:44,2018-02-09,-14.0,9.0
26156,afe0ea024346be7fa63b997fd149c757,842f5b7d1c537e492f0f039d881df246,delivered,2018-07-27 21:21:28,2018-07-27 21:32:01,2018-07-31 08:33:00,2018-08-06 18:41:35,2018-09-10,-35.0,9.0


### PULIZIA: df_order_review

In [68]:
# 1) Conversione della colonna review_creation_date in datetime
# 2) Droppiamo le colonne (review_comment_title, review_comment_message)

# 1
df_order_review['review_creation_date'] = pd.to_datetime(
    df_order_review['review_creation_date']
)

# 2
df_order_review = df_order_review.drop(columns= ['review_comment_title', 'review_comment_message'])

df_order_review.sample(2)

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
35256,e4b574976dab9988dad9a6f4510e1c77,1f039c377ad805f38658f64f2102c992,5,2018-08-16,2018-08-17 13:32:32
9768,7be4bca7b6e426e0d5a853ec14088244,863f8a1edc6ce924bde60328b112c402,5,2018-04-24,2018-04-25 00:11:28


### PULIZIA: df_order_payments

In [69]:
# Per un ordine ci sono diverse righe di pagamenti
# Aggreghiamo per order_id

df_order_payments = (
    df_order_payments
    .groupby('order_id', as_index=False)
    .agg(
        total_payment_value= ('payment_value', 'sum'),
        payment_installments= ('payment_installments', 'max'),
        payment_type         = ('payment_type', 'first')
    )
)

df_order_payments.sample(2)

,order_id,total_payment_value,payment_installments,payment_type
85554,dbf6633516ca4df83ac9e96e2100846d,85.84,8,credit_card
5202,0d4a3f1765974c4b0f1f07bb6ce1bda3,21.39,1,voucher


In [70]:
df_order_payments['eur_total_payment_value']=round(df_order_payments['total_payment_value']/EUR_BRL,2)#convesrione
df_order_payments=df_order_payments.drop(['total_payment_value'],axis=1)

In [71]:
df_order_payments.sample(2)

,order_id,payment_installments,payment_type,eur_total_payment_value
3943,0a0ae1b561eab534493c5588efc5169c,1,boleto,53.18
70392,b57410d258461ade68673ab75276e66d,1,credit_card,5.62


### PULIZIA:df_geolocation

In [72]:
# Coordinate del Brasile
brazil = {
    "lat_min": -34.0, "lat_max": 5.0,
    "lng_min": -75.0, "lng_max": -28.0
}

# Filtra coordinate fuori dal Brasile
df_geolocation_clean = df_geolocation_dataset[
    df_geolocation_dataset["geolocation_lat"].between(brazil["lat_min"], brazil["lat_max"]) &
    df_geolocation_dataset["geolocation_lng"].between(brazil["lng_min"], brazil["lng_max"])
].copy()

# Raggruppa per zip code (media lat/lng, primo valore per città e stato)
df_geolocation_dataset = (
    df_geolocation_clean
    .groupby('geolocation_zip_code_prefix', as_index=False)
    .agg(
        geolocation_lat   = ('geolocation_lat',   'mean'),
        geolocation_lng   = ('geolocation_lng',   'mean'),
        geolocation_city  = ('geolocation_city',  'first'),
        geolocation_state = ('geolocation_state', 'first')
    )
)

df_geolocation_dataset.info()
df_geolocation_dataset.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 19011 entries, 0 to 19010
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  19011 non-null  int64  
 1   geolocation_lat              19011 non-null  float64
 2   geolocation_lng              19011 non-null  float64
 3   geolocation_city             19011 non-null  str    
 4   geolocation_state            19011 non-null  str    
dtypes: float64(2), int64(1), str(2)
memory usage: 742.7 KB


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
17383,88355,-27.067730,-48.905913,brusque,SC
6634,22620,-23.009475,-43.334530,rio de janeiro,RJ


### PULIZIA: df_list_seller

In [73]:
df_list_seller=df_list_seller.rename(columns={'seller_zip_code_prefix':'geolocation_zip_code_prefix'})
df_list_seller.sample(3)

,seller_id,geolocation_zip_code_prefix,seller_city,seller_state
2136,ca77545ca4d2dfd1431bf61334e2fa3c,87020,maringa,PR
2387,61f159ef6da2d441951d2c0efa719362,29160,serra,ES
995,5b925e1d006e9476d738aa200751b73b,4567,sao paulo,SP


### PULIZIA: df_customers

In [74]:
df_customers=df_customers.rename(columns={'customer_zip_code_prefix':'geolocation_zip_code_prefix'})
df_customers.sample(3)

,customer_id,customer_unique_id,geolocation_zip_code_prefix,customer_city,customer_state
21956,c155fc0c74e24ef74f2f62258b8c4bbc,438aa2b994cbb567b3a42e7c23185eed,28930,arraial do cabo,RJ
56184,8ee3752ed676be5e448ebda2266e87e5,d669236ac05e274f48c19d950f806bae,13063,campinas,SP
57677,1441fb0aa631b791bcb461e549d7b7ce,ed70ee444e63fcf427bb3601d89b4057,24422,sao goncalo,RJ


### EXPORT: tutti i dataframe puliti nella cartella output

In [75]:
import os

# Crea la cartella output se non esiste
os.makedirs("output", exist_ok=True)

dataframes_to_export = {
    "list_product":        df_list_product,
    "orders_items":        df_orders_items,
    "order_dataset":       df_order_dataset,
    "order_review":        df_order_review,
    "order_payments":      df_order_payments,
    "geolocation_dataset": df_geolocation_dataset,
    "list_seller":         df_list_seller,
    "customers":           df_customers,
}

for name, df in dataframes_to_export.items():
    df.to_csv(f"output/{name}.csv", index=False)